In [1]:
import pandas as pd
import numpy as np
import tmdbsimple as tmdb
import json, os, time
from tqdm.notebook import tqdm_notebook

In [2]:
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

In [42]:
#assign data to variables
title_basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
title_akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
title_ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

In [43]:
# Open saved file and preview again
basics = pd.read_csv(title_basics_url,sep='\t', low_memory=False)
akas = pd.read_csv(title_akas_url,sep='\t', low_memory=False)
rating = pd.read_csv(title_ratings_url,sep='\t', low_memory=False)

In [45]:
akas.replace({'\\N':np.nan}, inplace = True)
basics.replace({'\\N':np.nan}, inplace = True)
rating.replace({'\\N':np.nan}, inplace = True)

In [46]:
#Exclude any movie with missing values for genre or runtime
basics.dropna(subset = ['runtimeMinutes', 'genres'], inplace = True)
basics['titleType'].value_counts()

tvEpisode       1065118
short            574863
movie            367365
video            175176
tvMovie           88361
tvSeries          86628
tvSpecial         16490
tvMiniSeries      16106
tvShort            9389
videoGame           298
Name: titleType, dtype: int64

In [48]:
#Include only full-length movies (titleType = "movie").
basics = basics[basics.titleType == 'movie']

basics["titleType"].value_counts()

movie    367365
Name: titleType, dtype: int64

In [12]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
1172,tt0001184,movie,Don Juan de Serrallonga,Don Juan de Serrallonga,0,1910,NaN,58,"Adventure,Drama"


In [13]:
#Include only fictional movies (not from documentary genre)
basics['genres'].value_counts()

Drama                           67991
Documentary                     62259
Comedy                          28919
Comedy,Drama                    10410
Drama,Romance                    9790
                                ...  
Adult,Drama,Musical                 1
Crime,Fantasy,Romance               1
Adult,Drama,War                     1
Documentary,Musical,Thriller        1
Crime,Fantasy,Sci-Fi                1
Name: genres, Length: 1384, dtype: int64

In [14]:
is_doc = basics['genres'].str.contains('documentary' , case=False)
basics = basics[~is_doc]

In [15]:
basics['genres'].value_counts()

Drama                        67991
Comedy                       28919
Comedy,Drama                 10410
Drama,Romance                 9790
Horror                        7368
                             ...  
Short,Thriller                   1
Comedy,Reality-TV,Romance        1
Biography,Music,Mystery          1
Adventure,Horror,Musical         1
Crime,Fantasy,Sci-Fi             1
Name: genres, Length: 1156, dtype: int64

In [16]:
#Include only movies that were released 2000 - 2021 (include 2000 and 2021)
basics = basics[(basics['startYear'] >= '2000') & (basics['startYear'] <= '2021')]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34791,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61091,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67637,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77931,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86768,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [17]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [18]:
#keep only US entries.
akas['region'].value_counts()

JP    3947737
FR    3947708
DE    3930843
IN    3876152
ES    3870774
       ...   
TV          1
NU          1
PW          1
NR          1
TC          1
Name: region, Length: 247, dtype: int64

In [19]:
akas = akas[akas.region == 'US']
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [20]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers

34791       True
61091       True
67637       True
77931      False
86768       True
           ...  
9195721     True
9195730     True
9195769    False
9195814     True
9195898    False
Name: tconst, Length: 136805, dtype: bool

In [21]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34791,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61091,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67637,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86768,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
93904,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
9195185,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
9195581,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
9195721,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
9195730,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [22]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79647 entries, 34791 to 9195814
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          79647 non-null  object
 1   titleType       79647 non-null  object
 2   primaryTitle    79647 non-null  object
 3   originalTitle   79647 non-null  object
 4   isAdult         79647 non-null  object
 5   startYear       79647 non-null  object
 6   endYear         0 non-null      object
 7   runtimeMinutes  79647 non-null  object
 8   genres          79647 non-null  object
dtypes: object(9)
memory usage: 6.1+ MB


In [23]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1346280 entries, 5 to 33060654
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1346280 non-null  object
 1   ordering         1346280 non-null  int64 
 2   title            1346280 non-null  object
 3   region           1346280 non-null  object
 4   language         3697 non-null     object
 5   types            963762 non-null   object
 6   attributes       44846 non-null    object
 7   isOriginalTitle  1344905 non-null  object
dtypes: int64(1), object(7)
memory usage: 92.4+ MB


In [24]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258682 entries, 0 to 1258681
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1258682 non-null  object 
 1   averageRating  1258682 non-null  float64
 2   numVotes       1258682 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.8+ MB


In [50]:
akas["language"].fillna("unknwn", inplace = True)
akas["types"].fillna("unknwn", inplace = True)
akas["attributes"].fillna("unknwn", inplace = True)
akas["isOriginalTitle"].fillna("unknwn", inplace = True)

In [51]:
basics = basics.drop(columns = ["endYear"])

In [52]:
with open('/Users/Yeajoon Yoo/codingdojo/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [53]:
tmdb.API_KEY =  login['api-key']

In [54]:
def get_movie_and_rating(movie_id):    
    
    movie = tmdb.Movies(movie_id)
    movie_info = movie.info()
    releases = movie.releases()

    for c in releases['countries']:
        if c['iso_3166_1' ] =='US':
            movie_info['certification'] = c['certification']
    return movie_info

In [55]:
#Avengers
test = get_movie_and_rating("tt0848228")
test

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 209.046,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [56]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_rating.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

In [57]:
YEARS_TO_GET =[2000,2001]

In [58]:
def read_and_fix_json(JSON_FILE):
   
    try: 
        previous_df =  pd.read_json(JSON_FILE)
    
    except:
        
        with open(JSON_FILE,'r+') as f:
            bad_json = f.read()
            
            first_char = bad_json[0]
            final_brackets = {'[':']', 
                           "{":"}"}
            final_char = final_brackets[first_char]
            
            if bad_json[-1] != final_char:
                good_json = bad_json[:-1]
                good_json+=final_char
            else:
                raise Exception('ERROR is not due to mismatched final bracket.')
            
            f.seek(0)
            f.write(good_json)
           
        previous_df =  pd.read_json(JSON_FILE)
        
    return previous_df

In [59]:
def get_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] =='US':
            movie_info['certification'] = c['certification']
    return movie_info

In [60]:
def write_json(new_data, filename):
    with open(filename,'r+') as file:
        file_data = json.load(file)
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        file.seek(0)
        json.dump(file_data, file)

In [61]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == False:
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    df = basics.loc[basics['startYear']==YEAR].copy()
    movie_ids = df['tconst'].copy()
    previous_df = read_and_fix_json(JSON_FILE)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]        

    for movie_id in tqdm_notebook(movie_ids_to_get, desc=f'Movies from {YEAR}', position=1, leave=True):
        try:
            temp = get_rating(movie_id)
            write_json(temp, JSON_FILE)
            time.sleep(0.02)
            
        except Exception as e:
            continue
            
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000: 0it [00:00, ?it/s]

Movies from 2001: 0it [00:00, ?it/s]

In [62]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
rating.to_csv("Data/title_rating.csv.gz",compression='gzip',index=False)